# CNN Trading Prediction

This notebook seeks to create a convolutional neural network that makes an expectation as to the sign of the next price return, in other words whether price is going to increase or decrease in the next time period.

The motivation for using a CNN is it's common application in feature extraction from image data, the hope would be that a CNN could extract abstract features from time series data.

In [ ]:
# Installing technical indicator library for the Average True Range
!pip install pandas_ta

import pandas as pd
import os
from collections import deque
import random
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
import pandas_ta as ta

In [ ]:
SEQ_LEN = 20

# Predicting one time period into the future 
PREDICT_HORIZON = 1

In [ ]:
def classify(current_close, future_close):
    if float(future_close) > float(current_close):
        return 1
    else:
        return 0



def preprocess(df):

    for col in df.columns: 
        col_end = col.rsplit('_', -1)[-1] # get the last word of the string
        if col != 'target':
            if col_end == 'volume':
                df[col] = (df[col].values - df[col].mean())/df[col].std() # standardizing volume (mean 0 , std 1)

            elif col_end == 'value':
                df[col] = (df[col].values - df[col].mean())/df[col].std() # standardizing indicator values (mean 0 , std 1)

            elif col_end == 'open' or col_end == 'high' or col_end == 'low' or col_end == 'close': 
                df[col] = (df[col].values - df[col].mean())/df[col].std() # standardizing price returns

    df.dropna(inplace=True)  


    sequential_data = []  
    prev_days = deque(maxlen=SEQ_LEN)  

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store each row excluding target
        if len(prev_days) == SEQ_LEN:  # 20 sequences 
            sequential_data.append([np.array(prev_days), i[-1]])  

    random.shuffle(sequential_data)  

    buys = []  
    sells = []  
    flats = []

    # ---------------------------------------------------------------------------
    for seq, target in sequential_data:  
        if target == 0:  # sell
            sells.append([seq, target])  
        if target == 1:  # flat
            flats.append([seq, target])  
        elif target == 2:  # buy
            buys.append([seq, target])  

    
    random.shuffle(buys)  
    random.shuffle(sells)  
    random.shuffle(flats)  
    
    # Balancing our dataset
    lower = min(len(buys), len(sells), len(flats))  

    buys = buys[:lower]  
    flats = flats[:lower]
    sells = sells[:lower] 

    sequential_data = buys+flats+sells  
    random.shuffle(sequential_data)  

    X = []
    y = []

    for seq, target in sequential_data: 
        X.append(seq)  
        y.append(target)  

    return np.array(X), y  



def preprocess_stats_df(main_df):
    
    df = main_df.copy()
    stats_dict = {}
                      
    for col in df.columns:
        col_end = col.rsplit('_', -1)[-1] # get the last word of the string
        if col_end == 'volume':
            mean = df[col].mean()
            std = df[col].std()
            stats_dict[f'{col}'] = {'mean': mean, 'std': std}

        elif col_end == 'value':
            mean = df[col].mean()
            std = df[col].std()
            stats_dict[f'{col}'] = {'mean': mean, 'std': std}

        elif col_end == 'open' or col_end == 'high' or col_end == 'low' or col_end == 'close':
            mean = df[col].mean()
            std = df[col].std()
            stats_dict[f'{col}'] = {'mean': mean, 'std': std}

    return stats_dict

In [ ]:
# Crypto hourly data
path = ''
main_df = pd.read_csv(path)
main_df.set_index('time', inplace=True)

main_df.sort_index() 

In [ ]:
# Time transformations

day = 60*60*24 # seconds in the day
week = 60*60*24*7 # seconds in week

main_df['Day_sin'] = np.sin(main_df.index * (2 * np.pi / day))
main_df['Day_cos'] = np.cos(main_df.index * (2 * np.pi / day))
main_df['Week_sin'] = np.sin(main_df.index * (2 * np.pi / week))
main_df['Week_cos'] = np.cos(main_df.index * (2 * np.pi / week))

In [ ]:
# Features

main_df['willr_value'] = ta.willr(close=main_df4['close'], high=main_df['high'], low=main_df4['low'])
main_df['nvi_value'] = ta.nvi(close=main_df['close'], volume=main_df['volume'])

for i in [1, 2, 3, 5, 10, 20]:    
    main_df[f'ret_{i}_close'] = main_df['close'].pct_change(i)
    main_df[f'ret_{i}_high'] = main_df['high'].pct_change(i)
    main_df[f'ret_{i}_low'] = main_df['low'].pct_change(i)
    main_df[f'ret_{i}_open'] = main_df['open'].pct_change(i)
    main_df[f'ret_{i}_volume'] = main_df['volume'].pct_change(i)

for i in [5, 10, 20, 40]:
    main_df[f'volt_{i}_close'] = np.log(1 + main_df['ret_1_close']).rolling(i).std()
    

main_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_df.dropna(inplace=True)

In [ ]:
# Stats dictionary for deployment (best practice to keep commented when not needing it)
# stats = preprocess_stats_df(main_df)
# stats

In [ ]:
# Creating column for future close
main_df['future_close_1'] = main_df['close'].shift(-PREDICT_HORIZON1)

main_df.dropna(inplace=True)
main_df

In [ ]:
# creating target column. Mapping classify function 
main_df['target'] = list(map(classify, main_df['close'], main_df['future_close_1']))

main_df = main_df.drop('future_close_1', axis=1)

main_df

In [ ]:
# Getting list of sorted times to split into train and validation sets
times = sorted(main_df.index.values)

main_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_df.dropna(inplace=True)
main_df

In [ ]:
# Calculating the last 20% threshold
last_20pct = times[-int(0.20*len(times))]

# Splitting into train and validation
validation_main_df = main_df[(main_df.index >= last_20pct)]
main_df = main_df[(main_df.index < last_20pct)]

# Preprocessing, X and y
train_x, train_y = preprocess(main_df)
validation_x, validation_y = preprocess(validation_main_df)

# Checking the outcome of the above
print(f'train data: {len(train_x)} validation: {len(validation_x)}')
print(f'positive returns: {train_y.count(1)}, negative returns: {train_y.count(0)})
print(f'VALIDATION positive returns: {validation_y.count(1)}, negative returns: {validation_y.count(0)})

In [ ]:
# Creating arrays for our neural network
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

### Pure CNN Architecture

In [ ]:
model = Sequential()

model.add(Conv1D(15, kernel_size=3, activation = 'tanh', input_shape=(train_x.shape[1:])))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6, amsgrad=True)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

filepath = 'NETWORK_NAME_{epoch:02d}'
checkpoint = ModelCheckpoint('save_path/{}.model'.format(save_path, filepath, monitor='val_loss', 
                                                  verbose=1, save_best_only=True, mode='min')) 

In [ ]:
history = model.fit(
    train_x, train_y,
    batch_size=32,
    epochs=100,
    validation_data=(validation_x, validation_y),
    callbacks=[checkpoint]
    )

### Alternate CNN LSTM Architecture

In [ ]:
model2 = Sequential()

model2.add(TimeDistributed(Conv1D(filters=20,
                                  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5),
                                  bias_regularizer=regularizers.L2(1e-5),
                                  activity_regularizer=regularizers.L2(1e-5),
                                  kernel_size=4, 
                                  activation='tanh'), input_shape=(None, 20, 9)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2, strides=None)))
model2.add(TimeDistributed(Flatten()))
model2.add(Dropout(0.1))


model2.add(Bidirectional(LSTM(32, 
                              kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5),
                              bias_regularizer=regularizers.L2(1e-5),
                              activity_regularizer=regularizers.L2(1e-5), 
                              activation='tanh', input_shape=(train_x.shape[1:]))))
model2.add(BatchNormalization())
model2.add(Dropout(0.1))

model2.add(Dense(16, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.1))

model2.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6, amsgrad=True)

# Compile model
model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

filepath = 'NETWORK_2_NAME_{epoch:02d}'
checkpoint = ModelCheckpoint('save_path/{}.model'.format(save_path, filepath, monitor='val_loss', 
                                                  verbose=1, save_best_only=True, mode='min')) 

In [ ]:
history = model2.fit(
    train_x, train_y,
    batch_size=32,
    epochs=100,
    validation_data=(validation_x, validation_y),
    callbacks=[checkpoint]
    )